# PIPEX

## Makefile

In [13]:
%%file ../pipex_working/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/01/30 14:27:11 by dpentlan          #+#    #+#              #
#    Updated: 2023/04/29 12:19:40 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = pipex
RED='\033[0;31m'
GREEN='\033[0;32m'
NOCOLOR='\033[0m'

# ***** Sources *****

SRCS = pipex.c \
	pipex_utils_1.c \

SRCS_DIR = srcs/

# ***** Includes *****

INCS = pipex.h

INCS_DIR = includes/

# ***** Objects *****

OBJS = $(patsubst %.c,$(OBJS_DIR)%.o,$(SRCS))

OBJS_DIR = objs/

# ***** Compile and Archiver *****

CFLAGS = -Wall -Wextra -Werror -g

CC = cc

LIBFT = -Llibft -lft

# ***** Build *****

all: $(NAME)

bonus: $(BONUS)

$(NAME): $(OBJS) $(OBJS_MAN)
	@echo -e -n ${GREEN}
	@make -C libft --silent
	@echo -e ===== MAKING PIPEX =====
	@echo -e -n ${NOCOLOR}
	@$(CC) $(CFLAGS) $(OBJS) $(LIBFT) -o $(NAME)

$(OBJS_DIR)%.o: $(SRCS_DIR)%.c
	@mkdir -p $(OBJS_DIR)
	@$(CC) -c $(CFLAGS) -I$(INCS_DIR) $< -o $@

clean:
	@make -C libft clean --silent
	@rm -f $(OBJS)
	@rm -rf $(OBJS_DIR)

fclean: clean
	@make -C libft fclean --silent
	@rm -f $(NAME)
	@rm -f gdb.txt

re: fclean all


Overwriting ../pipex_working/Makefile


## HEADERS

In [30]:
%%file ../pipex_working/includes/pipex.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.h                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:57:04 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/20 12:57:37 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef PIPEX_H
# define PIPEX_H

# include "../libft/includes/libft.h"
# include <unistd.h>
# include <sys/wait.h>

typedef struct	s_pipex {
	int 		fds[2];
	pid_t		pid1;
	pid_t		pid2;
}				t_pipex;

//	pipex.c
//	int main (int argc, char **argv, char **envp);

//	pipex_utils_1.c


#endif


Overwriting ../pipex_working/includes/pipex.h


## SOURCES

### pipex.c

In [33]:
%%file ../pipex_working/srcs/pipex.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.c                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:56:25 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/29 16:03:05 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	main(int argc, char **argv, char **envp)
{
	if (argc) {}
	if (argv) {}
	if (envp) {}

	t_pipex	*pipex;

	pipex = (t_pipex *)malloc(1 * sizeof(t_pipex));
	if (!pipex)
		return (1);
	if (pipe(pipex->fds))
		return (free(pipex), 1);

	pipex->pid1 = fork();
	if (pipex->pid1 == 0)
	{
		char *args[] = {"/usr/bin/ping", "ping", "-c", "5", "google.com", NULL};
		ft_printf("checking %s: result %d\n", args[0], access(args[0], F_OK | X_OK));
		dup2(pipex->fds[1], STDOUT_FILENO);
		close(pipex->fds[0]);
		close(pipex->fds[1]);
		execve(args[0], args + 1, NULL);
	}

	pipex->pid2 = fork();
	if (pipex->pid2 == 0)
	{
		char *args[] = {"/usr/bin/grep", "grep", "rtt", NULL};
		ft_printf("checking %s: result %d\n", args[0], access(args[0], F_OK | X_OK));
		dup2(pipex->fds[0], STDIN_FILENO);
		close(pipex->fds[0]);
		close(pipex->fds[1]);
		execve(args[0], args + 1, NULL);
	}

	close(pipex->fds[0]);
	close(pipex->fds[1]);
	waitpid(pipex->pid1, NULL, 0);
	waitpid(pipex->pid2, NULL, 0);
	free(pipex);
	return (0);
}


Overwriting ../pipex_working/srcs/pipex.c


### pipex_utils_1.c

In [16]:
%%file ../pipex_working/srcs/pipex_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_utils_1.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/29 17:20:54 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/29 17:21:02 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"



Overwriting ../pipex_working/srcs/pipex_utils_1.c


## TESTING

### Test Build

In [18]:
%%bash
cd ../pipex_working
make re

===== MAKING LIBFT =====
===== MAKING PIPEX =====


In [34]:
%%bash
cd ../pipex_working
make re > /dev/null
./pipex > /dev/null
nm -Duj pipex | grep -v "^_"
make fclean

access@GLIBC_2.2.5
close@GLIBC_2.2.5
dup2@GLIBC_2.2.5
execve@GLIBC_2.2.5
fork@GLIBC_2.2.5
free@GLIBC_2.2.5
malloc@GLIBC_2.2.5
pipe@GLIBC_2.2.5
waitpid@GLIBC_2.2.5
write@GLIBC_2.2.5


In [2]:
%%bash
# use valgrind with --track-fds=all to see if fds are closing appropriately
valgrind --track-fds=all < file1 cat | wc > file2

==86224== Memcheck, a memory error detector
==86224== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==86224== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==86224== Command: cat
==86224== 
==86224== 
==86224== FILE DESCRIPTORS: 0 open (0 std) at exit.
==86224== 
==86224== HEAP SUMMARY:
==86224==     in use at exit: 0 bytes in 0 blocks
==86224==   total heap usage: 1,051 allocs, 1,051 frees, 205,320 bytes allocated
==86224== 
==86224== All heap blocks were freed -- no leaks are possible
==86224== 
==86224== For lists of detected and suppressed errors, rerun with: -s
==86224== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)
